# Elastic Rod Model

In [1]:
from IPython.display import display, Markdown
def latexify(x):
    out = '$' + x + '$'
    return out

def lprint(x):
    display(Markdown(latexify(latex(x))))
    
%run -i 'implicit.py'

In this notebook a discretisation of the elastic rod bifurcation problem will be investigated

https://pubs.rsc.org/en/content/chapterhtml/2015/bk9781849738132-00001?isbn=978-1-84973-813-2&sercode=bk

derives a system of equations that correspond to a period truss network model for an elastic rod.

In [2]:
def get_funcs(n, k = 1, b = 1):
    # returns a set of equations to solve
    var_dict = {}
    for i in range(0,n+1):
        # create a grid
        # Pij is the displacement from the point (i,j) on the grid
        # then split into x,y position of where that point goes
        var_dict['p{}0x'.format(i)] = var('p{}0x'.format(i))
        var_dict['p{}0y'.format(i)] = var('p{}0y'.format(i))
        var_dict['p{}1x'.format(i)] = var('p{}1x'.format(i))
        var_dict['p{}1y'.format(i)] = var('p{}1y'.format(i))
        
    # enforce fixed points at either end of the truss network
    var_dict['p00x'] = 0
    var_dict['p00y'] = 0
    var_dict['p01x'] = 0
    var_dict['p01y'] = 0 # really 1
    
    #var_dict['p{}0x'.format(n)] = 0 # really n
    #var_dict['p{}0y'.format(n)] = 0
    #var_dict['p{}1x'.format(n)] = 0 # really n
    #var_dict['p{}1y'.format(n)] = 0 # really 1
    
    var_dict['l'] = var('l', latex_name=r'\lambda') # lamdba force variable
    
    # so after constraints at endpoints we have 4(n-1) uknowns
    
    b = 1
    k = 1
    
    funcs = []
    e_truss = 0
    for j in range(1,n+1):
        # iterate through each square in the truss
        # some values are used to generate the equations
            
        # xy displacements of the derived values
        ux = (var_dict['p{}0x'.format(j)] + var_dict['p{}1x'.format(j)])/2 - (var_dict['p{}0x'.format(j-1)] + var_dict['p{}1x'.format(j-1)])/2
        uy = (var_dict['p{}0y'.format(j)] + var_dict['p{}1y'.format(j)])/2 - (var_dict['p{}0y'.format(j-1)] + var_dict['p{}1y'.format(j-1)])/2 
               
                      
        vx = (var_dict['p{}1x'.format(j-1)] + var_dict['p{}1x'.format(j)])/2 - (var_dict['p{}0x'.format(j-1)] + var_dict['p{}0x'.format(j)])/2
        vy = (var_dict['p{}1y'.format(j-1)] + var_dict['p{}1y'.format(j)])/2 - (var_dict['p{}0y'.format(j-1)] + var_dict['p{}0y'.format(j)])/2
        
        wx = (var_dict['p{}0x'.format(j-1)] - var_dict['p{}0x'.format(j)] - var_dict['p{}1x'.format(j-1)] + var_dict['p{}1x'.format(j)])/2
        wy = (var_dict['p{}0y'.format(j-1)] - var_dict['p{}0y'.format(j)] - var_dict['p{}1y'.format(j-1)] + var_dict['p{}1y'.format(j)])/2
        
        q = 2*k*(ux*( b*(1+b)*ux + b*vy ) + vy*( b*ux + (1+b)*vy ))
        e1 = 2*k*b*((uy + vx)^2)
        e2 = 2*k*b*b*wx*wx
        e3 = 2*k*wy*wy
        
        e_truss = e_truss + (q + e1 + e2 + e3)/2
        
    e_truss = e_truss - (var_dict['l'] * (var_dict['p{}1x'.format(n)] + var_dict['p{}0x'.format(n)])^2)
    # add work done by a force lambda
        
    # solutions will be local minima of the energy equation
    for j in range(1,n+1):
        # only care about 1,...,n
        # 0 and n are fixed points
        funcs.append(e_truss.diff(var_dict['p{}0x'.format(j)]))
        funcs.append(e_truss.diff(var_dict['p{}0y'.format(j)]))
        funcs.append(e_truss.diff(var_dict['p{}1x'.format(j)]))
        funcs.append(e_truss.diff(var_dict['p{}1y'.format(j)]))
        
    return funcs, var_dict
    

In [53]:
funcs, var_dict = get_funcs(3)

In [54]:
lprint(funcs)

$ \left[4 \, \mathit{p10x} - \mathit{p20x} - \mathit{p21x} - \mathit{p21y}, 4 \, \mathit{p10y} - 2 \, \mathit{p11y} - \mathit{p21x} - \mathit{p21y}, 4 \, \mathit{p11x} - \mathit{p20x} + \mathit{p20y} - \mathit{p21x}, -2 \, \mathit{p10y} + 4 \, \mathit{p11y} + \mathit{p20x} - \mathit{p20y}, -\mathit{p10x} - \mathit{p11x} + \mathit{p11y} + 4 \, \mathit{p20x} - \mathit{p30x} - \mathit{p31x} - \mathit{p31y}, \mathit{p11x} - \mathit{p11y} + 4 \, \mathit{p20y} - 2 \, \mathit{p21y} - \mathit{p31x} - \mathit{p31y}, -\mathit{p10x} - \mathit{p10y} - \mathit{p11x} + 4 \, \mathit{p21x} - \mathit{p30x} + \mathit{p30y} - \mathit{p31x}, -\mathit{p10x} - \mathit{p10y} - 2 \, \mathit{p20y} + 4 \, \mathit{p21y} + \mathit{p30x} - \mathit{p30y}, -2 \, {\lambda} {\left(\mathit{p30x} + \mathit{p31x}\right)} - \mathit{p20x} - \mathit{p21x} + \mathit{p21y} + 2 \, \mathit{p30x} - \mathit{p30y}, \mathit{p21x} - \mathit{p21y} - \mathit{p30x} + 2 \, \mathit{p30y} - \mathit{p31y}, -2 \, {\lambda} {\left(\mathit{p30x} + \mathit{p31x}\right)} - \mathit{p20x} - \mathit{p20y} - \mathit{p21x} + 2 \, \mathit{p31x} + \mathit{p31y}, -\mathit{p20x} - \mathit{p20y} - \mathit{p30y} + \mathit{p31x} + 2 \, \mathit{p31y}\right] $

In [55]:
old_var = var_dict
var_dict = {key : var_dict[key] for key in var_dict.keys() if var_dict[key] != 0}
var_dict = {key : var_dict[key] for key in var_dict.keys() if key != 'l'}

J = jacobian(funcs,tuple(var_dict.values()))(**{var : 0 for var in var_dict.keys()})
lprint(J)

$ \left(\begin{array}{rrrrrrrrrrrr}
4 & 0 & 0 & 0 & -1 & 0 & -1 & -1 & 0 & 0 & 0 & 0 \\
0 & 4 & 0 & -2 & 0 & 0 & -1 & -1 & 0 & 0 & 0 & 0 \\
0 & 0 & 4 & 0 & -1 & 1 & -1 & 0 & 0 & 0 & 0 & 0 \\
0 & -2 & 0 & 4 & 1 & -1 & 0 & 0 & 0 & 0 & 0 & 0 \\
-1 & 0 & -1 & 1 & 4 & 0 & 0 & 0 & -1 & 0 & -1 & -1 \\
0 & 0 & 1 & -1 & 0 & 4 & 0 & -2 & 0 & 0 & -1 & -1 \\
-1 & -1 & -1 & 0 & 0 & 0 & 4 & 0 & -1 & 1 & -1 & 0 \\
-1 & -1 & 0 & 0 & 0 & -2 & 0 & 4 & 1 & -1 & 0 & 0 \\
0 & 0 & 0 & 0 & -1 & 0 & -1 & 1 & -2 \, {\lambda} + 2 & -1 & -2 \, {\lambda} & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 1 & -1 & -1 & 2 & 0 & -1 \\
0 & 0 & 0 & 0 & -1 & -1 & -1 & 0 & -2 \, {\lambda} & 0 & -2 \, {\lambda} + 2 & 1 \\
0 & 0 & 0 & 0 & -1 & -1 & 0 & 0 & 0 & -1 & 1 & 2
\end{array}\right) $

In [56]:
lprint(det(J))

$ -3748 \, {\lambda} + 485 $

Here we see that $\lambda = \frac{49}{248}$ will lead to degeneracy, so fix that and continue

In [57]:
bif_funcs = [func(l = 485/3748) for func in funcs] #
J = jacobian(bif_funcs,tuple(var_dict.values()))(**{var : 0 for var in var_dict.keys()})
lprint(J)

$ \left(\begin{array}{rrrrrrrrrrrr}
4 & 0 & 0 & 0 & -1 & 0 & -1 & -1 & 0 & 0 & 0 & 0 \\
0 & 4 & 0 & -2 & 0 & 0 & -1 & -1 & 0 & 0 & 0 & 0 \\
0 & 0 & 4 & 0 & -1 & 1 & -1 & 0 & 0 & 0 & 0 & 0 \\
0 & -2 & 0 & 4 & 1 & -1 & 0 & 0 & 0 & 0 & 0 & 0 \\
-1 & 0 & -1 & 1 & 4 & 0 & 0 & 0 & -1 & 0 & -1 & -1 \\
0 & 0 & 1 & -1 & 0 & 4 & 0 & -2 & 0 & 0 & -1 & -1 \\
-1 & -1 & -1 & 0 & 0 & 0 & 4 & 0 & -1 & 1 & -1 & 0 \\
-1 & -1 & 0 & 0 & 0 & -2 & 0 & 4 & 1 & -1 & 0 & 0 \\
0 & 0 & 0 & 0 & -1 & 0 & -1 & 1 & \frac{3263}{1874} & -1 & -\frac{485}{1874} & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 1 & -1 & -1 & 2 & 0 & -1 \\
0 & 0 & 0 & 0 & -1 & -1 & -1 & 0 & -\frac{485}{1874} & 0 & \frac{3263}{1874} & 1 \\
0 & 0 & 0 & 0 & -1 & -1 & 0 & 0 & 0 & -1 & 1 & 2
\end{array}\right) $

### Orthogonalising the kernel

In [58]:
K = J.right_kernel().basis()
lprint(K)

$ \left[\left(1,\,\frac{89}{287},\,1,\,-\frac{89}{287},\,\frac{614}{287},\,\frac{80}{287},\,\frac{614}{287},\,-\frac{80}{287},\,\frac{937}{287},\,\frac{81}{287},\,\frac{937}{287},\,-\frac{81}{287}\right)\right] $

In [59]:
Kperp = matrix(K).right_kernel().basis()
lprint(Kperp)

$ \left[\left(1,\,0,\,0,\,0,\,0,\,0,\,0,\,0,\,0,\,0,\,0,\,\frac{287}{81}\right), \left(0,\,1,\,0,\,0,\,0,\,0,\,0,\,0,\,0,\,0,\,0,\,\frac{89}{81}\right), \left(0,\,0,\,1,\,0,\,0,\,0,\,0,\,0,\,0,\,0,\,0,\,\frac{287}{81}\right), \left(0,\,0,\,0,\,1,\,0,\,0,\,0,\,0,\,0,\,0,\,0,\,-\frac{89}{81}\right), \left(0,\,0,\,0,\,0,\,1,\,0,\,0,\,0,\,0,\,0,\,0,\,\frac{614}{81}\right), \left(0,\,0,\,0,\,0,\,0,\,1,\,0,\,0,\,0,\,0,\,0,\,\frac{80}{81}\right), \left(0,\,0,\,0,\,0,\,0,\,0,\,1,\,0,\,0,\,0,\,0,\,\frac{614}{81}\right), \left(0,\,0,\,0,\,0,\,0,\,0,\,0,\,1,\,0,\,0,\,0,\,-\frac{80}{81}\right), \left(0,\,0,\,0,\,0,\,0,\,0,\,0,\,0,\,1,\,0,\,0,\,\frac{937}{81}\right), \left(0,\,0,\,0,\,0,\,0,\,0,\,0,\,0,\,0,\,1,\,0,\,1\right), \left(0,\,0,\,0,\,0,\,0,\,0,\,0,\,0,\,0,\,0,\,1,\,\frac{937}{81}\right)\right] $

In [60]:
A = matrix(K+Kperp).inverse().apply_map(lambda x: x.full_simplify())
lprint(A)

$ \left(\begin{array}{rrrrrrrrrrrr}
\frac{82369}{2716432} & \frac{2634063}{2716432} & -\frac{25543}{2716432} & -\frac{82369}{2716432} & \frac{25543}{2716432} & -\frac{88109}{1358216} & -\frac{1435}{169777} & -\frac{88109}{1358216} & \frac{1435}{169777} & -\frac{268919}{2716432} & -\frac{23247}{2716432} & -\frac{268919}{2716432} \\
\frac{25543}{2716432} & -\frac{25543}{2716432} & \frac{2708511}{2716432} & -\frac{25543}{2716432} & \frac{7921}{2716432} & -\frac{27323}{1358216} & -\frac{445}{169777} & -\frac{27323}{1358216} & \frac{445}{169777} & -\frac{83393}{2716432} & -\frac{7209}{2716432} & -\frac{83393}{2716432} \\
\frac{82369}{2716432} & -\frac{82369}{2716432} & -\frac{25543}{2716432} & \frac{2634063}{2716432} & \frac{25543}{2716432} & -\frac{88109}{1358216} & -\frac{1435}{169777} & -\frac{88109}{1358216} & \frac{1435}{169777} & -\frac{268919}{2716432} & -\frac{23247}{2716432} & -\frac{268919}{2716432} \\
-\frac{25543}{2716432} & \frac{25543}{2716432} & \frac{7921}{2716432} & \frac{25543}{2716432} & \frac{2708511}{2716432} & \frac{27323}{1358216} & \frac{445}{169777} & \frac{27323}{1358216} & -\frac{445}{169777} & \frac{83393}{2716432} & \frac{7209}{2716432} & \frac{83393}{2716432} \\
\frac{88109}{1358216} & -\frac{88109}{1358216} & -\frac{27323}{1358216} & -\frac{88109}{1358216} & \frac{27323}{1358216} & \frac{584859}{679108} & -\frac{3070}{169777} & -\frac{94249}{679108} & \frac{3070}{169777} & -\frac{287659}{1358216} & -\frac{24867}{1358216} & -\frac{287659}{1358216} \\
\frac{1435}{169777} & -\frac{1435}{169777} & -\frac{445}{169777} & -\frac{1435}{169777} & \frac{445}{169777} & -\frac{3070}{169777} & \frac{169377}{169777} & -\frac{3070}{169777} & \frac{400}{169777} & -\frac{4685}{169777} & -\frac{405}{169777} & -\frac{4685}{169777} \\
\frac{88109}{1358216} & -\frac{88109}{1358216} & -\frac{27323}{1358216} & -\frac{88109}{1358216} & \frac{27323}{1358216} & -\frac{94249}{679108} & -\frac{3070}{169777} & \frac{584859}{679108} & \frac{3070}{169777} & -\frac{287659}{1358216} & -\frac{24867}{1358216} & -\frac{287659}{1358216} \\
-\frac{1435}{169777} & \frac{1435}{169777} & \frac{445}{169777} & \frac{1435}{169777} & -\frac{445}{169777} & \frac{3070}{169777} & \frac{400}{169777} & \frac{3070}{169777} & \frac{169377}{169777} & \frac{4685}{169777} & \frac{405}{169777} & \frac{4685}{169777} \\
\frac{268919}{2716432} & -\frac{268919}{2716432} & -\frac{83393}{2716432} & -\frac{268919}{2716432} & \frac{83393}{2716432} & -\frac{287659}{1358216} & -\frac{4685}{169777} & -\frac{287659}{1358216} & \frac{4685}{169777} & \frac{1838463}{2716432} & -\frac{75897}{2716432} & -\frac{877969}{2716432} \\
\frac{23247}{2716432} & -\frac{23247}{2716432} & -\frac{7209}{2716432} & -\frac{23247}{2716432} & \frac{7209}{2716432} & -\frac{24867}{1358216} & -\frac{405}{169777} & -\frac{24867}{1358216} & \frac{405}{169777} & -\frac{75897}{2716432} & \frac{2709871}{2716432} & -\frac{75897}{2716432} \\
\frac{268919}{2716432} & -\frac{268919}{2716432} & -\frac{83393}{2716432} & -\frac{268919}{2716432} & \frac{83393}{2716432} & -\frac{287659}{1358216} & -\frac{4685}{169777} & -\frac{287659}{1358216} & \frac{4685}{169777} & -\frac{877969}{2716432} & -\frac{75897}{2716432} & \frac{1838463}{2716432} \\
-\frac{23247}{2716432} & \frac{23247}{2716432} & \frac{7209}{2716432} & \frac{23247}{2716432} & -\frac{7209}{2716432} & \frac{24867}{1358216} & \frac{405}{169777} & \frac{24867}{1358216} & -\frac{405}{169777} & \frac{75897}{2716432} & \frac{6561}{2716432} & \frac{75897}{2716432}
\end{array}\right) $

In [61]:
key_list = [str(var) for var in var_dict.values()]
value_list = list(A*vector(var_dict.values()))
mutate_dict = dict(zip(key_list, value_list))

In [62]:
Afuncs = [func(**mutate_dict) for func in bif_funcs]

In [63]:
lprint(Afuncs) # they don't change much, many cancellations

$ \left[4 \, \mathit{p10y} - \mathit{p20y} - \mathit{p21y} - \mathit{p30x}, 4 \, \mathit{p11x} - 2 \, \mathit{p20x} - \mathit{p21y} - \mathit{p30x}, 4 \, \mathit{p11y} - \mathit{p20y} + \mathit{p21x} - \mathit{p21y}, -2 \, \mathit{p11x} + 4 \, \mathit{p20x} + \mathit{p20y} - \mathit{p21x}, -\mathit{p10y} - \mathit{p11y} + \mathit{p20x} + 4 \, \mathit{p20y} - \mathit{p30y} - \mathit{p31y}, \mathit{p11y} - \mathit{p20x} + 4 \, \mathit{p21x} - 2 \, \mathit{p30x} - \mathit{p31y}, -\mathit{p10y} - \mathit{p11x} - \mathit{p11y} + 4 \, \mathit{p21y} - \mathit{p30y} + \mathit{p31x} - \mathit{p31y}, -\mathit{p10y} - \mathit{p11x} - 2 \, \mathit{p21x} + 4 \, \mathit{p30x} + \mathit{p30y} - \mathit{p31x}, -\mathit{p20y} - \mathit{p21y} + \mathit{p30x} + \frac{3263}{1874} \, \mathit{p30y} - \mathit{p31x} - \frac{485}{1874} \, \mathit{p31y}, \mathit{p21y} - \mathit{p30x} - \mathit{p30y} + 2 \, \mathit{p31x}, -\mathit{p20y} - \mathit{p21x} - \mathit{p21y} - \frac{485}{1874} \, \mathit{p30y} + \frac{3263}{1874} \, \mathit{p31y}, -\mathit{p20y} - \mathit{p21x} - \mathit{p31x} + \mathit{p31y}\right] $

In [64]:
Ja = jacobian(Afuncs,tuple(var_dict.values()))(**{var : 0 for var in var_dict.keys()}).apply_map(lambda x: x.full_simplify())
lprint(Ja)

$ \left(\begin{array}{rrrrrrrrrrrr}
0 & 4 & 0 & 0 & 0 & -1 & 0 & -1 & -1 & 0 & 0 & 0 \\
0 & 0 & 4 & 0 & -2 & 0 & 0 & -1 & -1 & 0 & 0 & 0 \\
0 & 0 & 0 & 4 & 0 & -1 & 1 & -1 & 0 & 0 & 0 & 0 \\
0 & 0 & -2 & 0 & 4 & 1 & -1 & 0 & 0 & 0 & 0 & 0 \\
0 & -1 & 0 & -1 & 1 & 4 & 0 & 0 & 0 & -1 & 0 & -1 \\
0 & 0 & 0 & 1 & -1 & 0 & 4 & 0 & -2 & 0 & 0 & -1 \\
0 & -1 & -1 & -1 & 0 & 0 & 0 & 4 & 0 & -1 & 1 & -1 \\
0 & -1 & -1 & 0 & 0 & 0 & -2 & 0 & 4 & 1 & -1 & 0 \\
0 & 0 & 0 & 0 & 0 & -1 & 0 & -1 & 1 & \frac{3263}{1874} & -1 & -\frac{485}{1874} \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & -1 & -1 & 2 & 0 \\
0 & 0 & 0 & 0 & 0 & -1 & -1 & -1 & 0 & -\frac{485}{1874} & 0 & \frac{3263}{1874} \\
0 & 0 & 0 & 0 & 0 & -1 & -1 & 0 & 0 & 0 & -1 & 1
\end{array}\right) $

In [65]:
Ka = Ja.right_kernel().basis()
lprint(Ka)

$ \left[\left(1,\,0,\,0,\,0,\,0,\,0,\,0,\,0,\,0,\,0,\,0,\,0\right)\right] $

### Orthogonalising the image

In [66]:
I = Ja.image().basis()
I = [Ja*vec for vec in I]
print(len(I))

11


In [67]:
Iperp = matrix(I).right_kernel().basis()
lprint(Iperp)

$ \left[\left(1,\,\frac{89}{287},\,1,\,-\frac{89}{287},\,\frac{614}{287},\,\frac{80}{287},\,\frac{614}{287},\,-\frac{80}{287},\,\frac{937}{287},\,\frac{81}{287},\,\frac{937}{287},\,-\frac{81}{287}\right)\right] $

In [68]:
B = matrix(I + Iperp).inverse().apply_map(lambda x: x.full_simplify())
lprint(B)

$ \left(\begin{array}{rrrrrrrrrrrr}
\frac{527035427}{55007748} & \frac{361007485}{110015496} & \frac{472027679}{55007748} & -\frac{250991989}{110015496} & \frac{2162555455}{110015496} & \frac{871076075}{220030992} & \frac{2052539959}{110015496} & -\frac{210983099}{220030992} & \frac{814781993}{27503874} & 5 & \frac{787278119}{27503874} & \frac{82369}{2716432} \\
\frac{613765129}{55007748} & \frac{483017507}{110015496} & \frac{558757381}{55007748} & -\frac{262986515}{110015496} & \frac{2562807161}{110015496} & \frac{1089010573}{220030992} & \frac{2452791665}{110015496} & -\frac{208886605}{220030992} & \frac{969428221}{27503874} & 6 & \frac{941924347}{27503874} & \frac{25543}{2716432} \\
-\frac{502492301}{55007748} & -\frac{353414131}{110015496} & -\frac{447484553}{55007748} & \frac{243398635}{110015496} & -\frac{2117183161}{110015496} & -\frac{875589701}{220030992} & -\frac{2007167665}{110015496} & \frac{215496725}{220030992} & -\frac{803532371}{27503874} & -5 & -\frac{776028497}{27503874} & \frac{82369}{2716432} \\
\frac{610666595}{55007748} & \frac{462300829}{110015496} & \frac{555658847}{55007748} & -\frac{242269837}{110015496} & \frac{2550876079}{110015496} & \frac{1095000755}{220030992} & \frac{2440860583}{110015496} & -\frac{214876787}{220030992} & \frac{967048241}{27503874} & 6 & \frac{939544367}{27503874} & -\frac{25543}{2716432} \\
\frac{409703317}{27503874} & \frac{273288533}{55007748} & \frac{382199443}{27503874} & -\frac{218280785}{55007748} & \frac{1750372667}{55007748} & \frac{659131759}{110015496} & \frac{1640357171}{55007748} & -\frac{219069775}{110015496} & \frac{657755785}{13751937} & 8 & \frac{630251911}{13751937} & \frac{88109}{1358216} \\
\frac{508635689}{13751937} & \frac{178718171}{13751937} & \frac{467379878}{13751937} & -\frac{123710423}{13751937} & \frac{1072329416}{13751937} & \frac{220015337}{13751937} & \frac{1017321668}{13751937} & -\frac{54992093}{13751937} & \frac{1622424046}{13751937} & 20 & \frac{1567416298}{13751937} & \frac{1435}{169777} \\
-\frac{397416541}{27503874} & -\frac{265244111}{55007748} & -\frac{369912667}{27503874} & \frac{210236363}{55007748} & -\frac{1695421325}{55007748} & -\frac{650188465}{110015496} & -\frac{1585405829}{55007748} & \frac{210126481}{110015496} & -\frac{642396481}{13751937} & -8 & -\frac{614892607}{13751937} & \frac{88109}{1358216} \\
\frac{507140102}{13751937} & \frac{178492637}{13751937} & \frac{465884291}{13751937} & -\frac{123484889}{13751937} & \frac{1067539892}{13751937} & \frac{216651107}{13751937} & \frac{1012532144}{13751937} & -\frac{51627863}{13751937} & \frac{1614204682}{13751937} & 20 & \frac{1559196934}{13751937} & -\frac{1435}{169777} \\
\frac{895858693}{55007748} & \frac{593631419}{110015496} & \frac{840850945}{55007748} & -\frac{483615923}{110015496} & \frac{3832625825}{110015496} & \frac{1415043757}{220030992} & \frac{3612594833}{110015496} & -\frac{534919789}{220030992} & \frac{1465851871}{27503874} & 9 & \frac{1383340249}{27503874} & \frac{268919}{2716432} \\
\frac{3800214257}{55007748} & \frac{2601880075}{110015496} & \frac{3525175517}{55007748} & -\frac{1941787099}{110015496} & \frac{16111269097}{110015496} & \frac{6280137173}{220030992} & \frac{15231145129}{110015496} & -\frac{1879517333}{220030992} & \frac{6105582929}{27503874} & 38 & \frac{5858048063}{27503874} & \frac{23247}{2716432} \\
-\frac{913285019}{55007748} & -\frac{604315093}{110015496} & -\frac{858277271}{55007748} & \frac{494299597}{110015496} & -\frac{3892906783}{110015496} & -\frac{1420911251}{220030992} & -\frac{3672875791}{110015496} & \frac{540787283}{220030992} & -\frac{1480118633}{27503874} & -9 & -\frac{1397607011}{27503874} & \frac{268919}{2716432} \\
\frac{3881176331}{55007748} & \frac{2651699413}{110015496} & \frac{3606137591}{55007748} & -\frac{1991606437}{110015496} & \frac{16458750583}{110015496} & \frac{6378435947}{220030992} & \frac{15578626615}{110015496} & -\frac{1977816107}{220030992} & \frac{6232111661}{27503874} & 38 & \frac{5984576795}{27503874} & -\frac{23247}{2716432}
\end{array}\right) $

In [69]:
tfuncs = vector(Afuncs)*B

In [70]:
lprint(tfuncs)

$ \left(\mathit{p10y},\,\mathit{p11x},\,\mathit{p11y},\,\mathit{p20x},\,\mathit{p20y},\,\mathit{p21x},\,\mathit{p21y},\,\mathit{p30x},\,\mathit{p30y},\,\mathit{p31x},\,\mathit{p31y},\,0\right) $

In [71]:
Jt = jacobian(tfuncs,tuple(var_dict.values()))(**{var : 0 for var in var_dict.keys()}).apply_map(lambda x: x.full_simplify())

In [72]:
lprint(Jt)

$ \left(\begin{array}{rrrrrrrrrrrr}
0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0
\end{array}\right) $

In [73]:
It = Jt.image().basis()
It = [Jt*vec for vec in It]
lprint(It)

$ \left[\left(1,\,0,\,0,\,0,\,0,\,0,\,0,\,0,\,0,\,0,\,0,\,0\right), \left(0,\,1,\,0,\,0,\,0,\,0,\,0,\,0,\,0,\,0,\,0,\,0\right), \left(0,\,0,\,1,\,0,\,0,\,0,\,0,\,0,\,0,\,0,\,0,\,0\right), \left(0,\,0,\,0,\,1,\,0,\,0,\,0,\,0,\,0,\,0,\,0,\,0\right), \left(0,\,0,\,0,\,0,\,1,\,0,\,0,\,0,\,0,\,0,\,0,\,0\right), \left(0,\,0,\,0,\,0,\,0,\,1,\,0,\,0,\,0,\,0,\,0,\,0\right), \left(0,\,0,\,0,\,0,\,0,\,0,\,1,\,0,\,0,\,0,\,0,\,0\right), \left(0,\,0,\,0,\,0,\,0,\,0,\,0,\,1,\,0,\,0,\,0,\,0\right), \left(0,\,0,\,0,\,0,\,0,\,0,\,0,\,0,\,1,\,0,\,0,\,0\right), \left(0,\,0,\,0,\,0,\,0,\,0,\,0,\,0,\,0,\,1,\,0,\,0\right), \left(0,\,0,\,0,\,0,\,0,\,0,\,0,\,0,\,0,\,0,\,1,\,0\right)\right] $

In [74]:
var_dict_old = var_dict
var_dict_old

{'p10x': p10x,
 'p10y': p10y,
 'p11x': p11x,
 'p11y': p11y,
 'p20x': p20x,
 'p20y': p20y,
 'p21x': p21x,
 'p21y': p21y,
 'p30x': p30x,
 'p30y': p30y,
 'p31x': p31x,
 'p31y': p31y}

In [25]:
lprint(tfuncs)

$ \left(\mathit{p10y},\,\mathit{p11x},\,\mathit{p11y},\,\mathit{p20x},\,\mathit{p20y},\,\mathit{p21x},\,\mathit{p21y},\,0\right) $

We see that this trivially has solution for every possible value in the kernel, thus bifurcation methods are unecessary

In [26]:
var_dict.values()

dict_values([p10x, p10y, p11x, p11y, p20x, p20y, p21x, p21y])

In [27]:
position = {key : 0 for key in var_dict_old.keys()}
position

{'p10x': 0,
 'p10y': 0,
 'p11x': 0,
 'p11y': 0,
 'p20x': 0,
 'p20y': 0,
 'p21x': 0,
 'p21y': 0}

In [45]:
#code_funcs = [tfuncs[index].function(*list(var_dict_old.values())) for index in [0,1,2,3,4,5,6]] # careful with computer science indexing from zero
code_funcs = [tfuncs[index] for index in [0,1,2,3,4,5,6]]
position = {key : 0 for key in var_dict_old.keys()}
# var_dict already loaded
var_dict = {'y1' : p10y, 'y2' : p11x, 'y3' : p11y, 'y4' : p20x, 'y5' : p20y, 'y6' : p21x, 'y7': p21y, 'x1' : p10x}
x_var_keys = [list(var_dict.keys())[index] for index in [7] ] # the variables in the kernel
x_dim = 1
y_dim = 7 # invertible part

In [46]:
code_funcs

[p10y, p11x, p11y, p20x, p20y, p21x, p21y]

In [47]:
var_dict.keys()

dict_keys(['y1', 'y2', 'y3', 'y4', 'y5', 'y6', 'y7', 'x1'])

In [48]:
t_dict = TensorDict(code_funcs, position, var_dict, x_dim, y_dim)

computing Taylor approximaton to 3 order for speedup


In [51]:
out = get_hkx_polynomial(code_funcs, 1, x_dim, y_dim, var_dict, x_var_keys, t_dict, position)

100%|██████████| 1/1 [00:00<00:00, 270.53it/s]

[1 0 0 0 0 0 0]
[0 1 0 0 0 0 0]
[0 0 1 0 0 0 0]
[0 0 0 1 0 0 0]
[0 0 0 0 1 0 0]
[0 0 0 0 0 1 0]
[0 0 0 0 0 0 1]


In [52]:
out

[0, 0, 0, 0, 0, 0, 0]